In [14]:
import models
from  models.simplenet import SimpleNet
import torch.optim as optim
import torch.nn as nn


model = SimpleNet(num_classes=10, in_chans= 1, scale=1, network_idx=1, mode=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [19]:
import torchvision
import torchvision.transforms as transforms
import os
import torch

# Define the transformation to apply to the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
notebook_directory = os.path.dirname(os.path.abspath("deep_ensemble_mnist.ipynb"))

# Navigate to the parent folder (assuming "usr" and "home" are at the same level)
parent_directory = os.path.abspath(os.path.join(notebook_directory, "..","..","..","..","..","..",".."))
# Construct the path to the ImageNet directory
mnist_directory = os.path.abspath(os.path.join(parent_directory, "/usr/data1/vision/data/"))
# Download the training dataset
train_dataset = torchvision.datasets.MNIST(root=mnist_directory, train=True, download=False, transform=transform)

# Download the testing dataset
test_dataset = torchvision.datasets.MNIST(root=mnist_directory, train=False, download=False, transform=transform)

train_loader128 = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=128,
        shuffle=True)

test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=64)

In [22]:
num_epochs = 10
for epoch in range(num_epochs):
    # Training
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch + 1} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

    # Testing
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset) * 100
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({accuracy:.2f}%)')

print("Training complete!")


Training complete!


In [20]:
import torch
from torch.utils.data import DataLoader
import push.bayes.ensemble

#load densenet

# densenet = densenet121(pretrained=False).cuda()
# densenet.eval()

epochs = 1
two_particle_params = push.bayes.ensemble.train_deep_ensemble(
        train_loader128,
        torch.nn.CrossEntropyLoss(),
        epochs,
        SimpleNet, 10, 1, 1, 1, 2,
        num_devices=2,
        num_ensembles=2,
    )

100%|██████████| 1/1 [00:33<00:00, 33.22s/it]
